In [58]:
import pandas as pd 
from sklearn.tree import plot_tree
import matplotlib.pyplot as plt
from sklearn import tree
from sklearn.model_selection import train_test_split

#1. Скачайте данные по ссылке
data=pd.read_csv("/content/titanic.csv")#
print("\n1. Скачали и Загрузили данные:: " )
print(data.head(5)) #|PassengerId|Survived|Pclass|Name|Sex|Age|SibSp|Parch|Ticket|Fare|Cabin|Embarked|


#2. Исключите признаки, которые на ваш взгляд, могут привести к переобучению (например, id).
data.drop(["PassengerId", "Name","SibSp","Parch","Ticket","Cabin",#"Fare","Embarked"
          ], axis="columns",inplace=True)
#print(data.head())

X = data.drop(["Survived"],axis="columns")
Y = data["Survived"]
#print(Y)

X.Sex = X.Sex.map({"male":0,"female":1})
#print(X.head())
#print(X.Age[0:10])

X.Age = X.Age.fillna(X.Age.mean())
#print(X.Age[0:10])
print("\n2. Исключили лишние признаки:: " )
print(X.head()) #|Pclass|Sex|Age|

#3. Разделите выборку на train и test.
X_train,X_test,Y_train,Y_real=train_test_split(X,Y,train_size = 0.8)
print("\n3. Разделили выборку:: " )
print(len(X_train))
print(len(X_test))
#print(X_test)


#4. Преобразуйте категориальные признаки с помощью sklearn.preprocessing.OneHotEncoder, 
#а некатегориальные с помощью sklearn.preprocessing.StandardScaler
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import make_column_transformer, ColumnTransformer

ct = ColumnTransformer([('scaling', StandardScaler(), ['Age', 'Fare']),
                       ("onehot", OneHotEncoder(sparse=False), ['Sex', 'Pclass', 'Embarked'])])

ct.fit(X_train)
X_train = ct.transform(X_train)
#dfX = pd.DataFrame(X_traint, columns = ['Age', 'Fare','Sex', 'Pclass', 'Embarked'])
#print(dfX.head())
X_test = ct.transform(X_test)
#print(X_testt)
print("\n4. Преобразовали категориальные и некатегориальные признаки:: " )
print(X_train)



1. Скачали и Загрузили данные:: 
   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            3

In [59]:
#5. Обучите модель логистической регрессии и посчитайте все метрики классификации
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
model_lr = LogisticRegression(solver='liblinear') #C=100, random_state=SEED
model_lr.fit(X_train,Y_train)

y_pred_lg = model_lr.predict(X_test)
#print(y_pred_lg)

print("\n5. Обучили модель логистической регрессии:: " )
print("Метрики для модели лог.регрессии:: " )
accuracy_lg = model_lr.score(X_test,Y_real) 
print(accuracy_lg)

acc_lg = metrics.accuracy_score(Y_real,y_pred_lg)
print(acc_lg)

precision_lg = metrics.precision_score(Y_real,y_pred_lg)  
print(precision_lg)

rec_lg = metrics.recall_score(Y_real, y_pred_lg)
print(rec_lg)

cm_lg = metrics.confusion_matrix(Y_real,y_pred_lg)
print(cm_lg)

auc_lg = metrics.roc_auc_score(Y_real, y_pred_lg)
print(auc_lg)



5. Обучили модель логистической регрессии:: 
Метрики для модели лог.регрессии:: 
0.8044692737430168
0.8044692737430168
0.7792207792207793
0.7692307692307693
[[84 17]
 [18 60]]
0.8004569687738005


In [60]:
#5. Обучите модель KNN и посчитайте все метрики классификации
#from sklearn.neighbors import KNeighborsRegressor
from sklearn.neighbors import KNeighborsClassifier
#from sklearn import metrics
import numpy as np

knn_model = KNeighborsClassifier() #(n_neighbors=3,p=2,metric='minkowski'
#print(knn_model)

knn_model.fit(X_train, Y_train)
y_pred_knn = knn_model.predict(X_test)
#print(y_pred_knn)

print("\n5. Обучили модель KNN:: " )
print("Метрики для модели KNN:: " )
rounding = np.round(y_pred_knn)
acc_knn = metrics.accuracy_score(Y_real,rounding)  
print(acc_knn)

precision_knn = metrics.precision_score(Y_real,rounding)  
print(precision_knn)

recall_knn = metrics.recall_score(Y_real, rounding)
print(recall_knn)

cm_knn = metrics.confusion_matrix(Y_real, rounding)
print(cm_knn)

auc_knn = metrics.roc_auc_score(Y_real, rounding)
print(auc_knn)



5. Обучили модель KNN:: 
Метрики для модели KNN:: 
0.7932960893854749
0.7887323943661971
0.717948717948718
[[86 15]
 [22 56]]
0.7847169332317847


In [61]:
#6. Подберите гиперпараметры для моделей с помощью RandomizedSearchCV
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

knn_params = {
    'n_neighbors': [3,5,6,7,10],
    'algorithm': ['auto' , 'ball_tree' , 'kd_tree' ,'brute'],
    'weights' : ['uniform' , 'distance'],
    'leaf_size': [10,20,30,40,50],
    'p': [1, 2, 3],
    'metric' : ['minkowski','euclidean','manhattan']
  }

lr_params = {"penalty": ["l1", "l2"],
         "tol": np.linspace(0.0001, 1.0, 50),
         "C": np.linspace(0.1, 1.0, 50),
         "class_weight": ["balanced", {0:0.8, 1:0.2}]}

rs = RandomizedSearchCV(estimator=knn_model, 
                        param_distributions=knn_params, 
                        n_iter=200, 
                        cv=5, 
                        verbose=3, 
                        random_state=0, 
                        n_jobs=-1)

rs.fit(X_train, Y_train)
#print(rs.best_score_)
#print(rs.best_params_)
#bestmodel = rs.best_estimator_
print("\n6. Подобрали Наилучшие гиперпараметры для модели KNN:: " )
print("Наилучшие гиперпараметры: {}".format(rs.best_params_))
print("Лучшее значение кросс-валидации: {:.2%}".format(rs.best_score_))
print("Лучшая модель:\n{}".format(rs.best_estimator_))


#7. Посчитайте метрики для новых моделей, также сделайте выводы.
#from sklearn import metrics
y_predict = rs.predict(X_test)

print("\n7. Посчитали метрики для модели KNN с наилучшими гиперпараметрами:: " )
print(f"Accuracy: {metrics.accuracy_score(Y_real, y_predict):.4%}")
print(f"Precision: {metrics.precision_score(Y_real, y_predict):.4%}")
print(f"Recall: {metrics.recall_score(Y_real, y_predict):.4%}")
print(f"ROC-AuC Score: {metrics.roc_auc_score(Y_real, rounding):.4%}")
cm = metrics.confusion_matrix(Y_real, y_predict)
print(cm)
print(f"F1 Score: {metrics.f1_score(Y_real, y_predict):.4%}")


Fitting 5 folds for each of 200 candidates, totalling 1000 fits

6. Подобрали Наилучшие гиперпараметры для модели KNN:: 
Наилучшие гиперпараметры: {'weights': 'uniform', 'p': 3, 'n_neighbors': 10, 'metric': 'manhattan', 'leaf_size': 50, 'algorithm': 'ball_tree'}
Лучшее значение кросс-валидации: 81.32%
Лучшая модель:
KNeighborsClassifier(algorithm='ball_tree', leaf_size=50, metric='manhattan',
                     n_neighbors=10, p=3)

7. Посчитали метрики для модели KNN с наилучшими гиперпараметрами:: 
Accuracy: 78.2123%
Precision: 85.4545%
Recall: 60.2564%
ROC-AuC Score: 78.4717%
[[93  8]
 [31 47]]
F1 Score: 70.6767%
